In [ ]:
import pandas as pd
import numpy as np
import os
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.metrics import precision_score, recall_score, f1_score, accuracy_score, confusion_matrix

def get_tpr_fpr(probas, ytestclass, i):
    probs = pd.DataFrame(list(zip(probas[:,i], ytestclass==i)), columns=['probability','ytrue'])
    function = lambda x: float(str(x).split(",")[1].split("]")[0])
    percentiles = pd.Series(pd.qcut(probs['probability'], q=100).sort_values(ascending=False).unique()).apply(function)
    Ts = probs[probs['ytrue']]['probability']
    Fs = probs[~probs['ytrue']]['probability']
    tpr = []
    fpr = []
    for j in percentiles.values:
        tpr.append(len(Ts[Ts>j]) / len(Ts))
        fpr.append(len(Fs[Fs>j]) / len(Fs))
    return np.array(tpr), np.array(fpr), percentiles.values

def get_auc_score(tpr,fpr):
    return np.trapz(tpr,fpr)

def get_roc_auc_curves(outputs, label):
    curves = pd.DataFrame(index=range(100))
    for i in range(outputs.shape[1]):
        curves[str(i)+'_tpr'],curves[str(i)+'_fpr'], _ = get_tpr_fpr(outputs, label, i)
    return curves

def get_weighted_auc_score(curves, label):
    scores = []
    for i in list(set(label)):
        multiplier = len(label[label==i]) / len(label)
        score = get_auc_score(curves[str(i)+'_tpr'],curves[str(i)+'_fpr'])
        scores.append(multiplier * score)
    return np.sum(scores)

def get_weighted_curve(curves, label):
    curves['tpr'], curves['fpr'] = 0,0
    for i in range(10):
        multiplier = len(label[label==i]) / len(label)
        curves['tpr'] += curves[str(i)+"_tpr"] * multiplier 
        curves['fpr'] += curves[str(i)+"_fpr"] * multiplier 
    return curves[['tpr','fpr']]